In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import time
import joblib

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df = pd.read_csv("dados/df_final.csv")
df.head(2)

,codigo_condidato,codigo_vaga,perfil_compativel,titulo_vaga,cliente,tipo_contratacao,vaga_especifica_para_pcd,nivel profissional,nivel_academico,nivel_ingles,principais_atividades,competencia_tecnicas_e_comportamentais,cv_pt
0,25632,4530,sim,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,\ndados pessoais\nestado civil: casado\nidade:...
1,25529,4530,sim,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,"solteiro, 47 anos\n\nestrada meringuava, nº 17..."


## Pré-processamento dos dados

#### Função que remove pontuação, converte para minúsculas e remove stopwords

In [3]:
def preprocess_text(text):
    if not isinstance(text, str) or not text.strip():
        return ''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

#### Combina a informação de todas as colunas referentes a vaga em um único texto

In [4]:
def combine_vaga_info(row):
    fields = ['titulo_vaga', 'principais_atividades', 'competencia_tecnicas_e_comportamentais', 'tipo_contratacao',	
              'vaga_especifica_para_pcd', 'nivel profissional',	'nivel_academico', 'nivel_ingles']
    texts = [str(row.get(field, '')) for field in fields]
    return ' '.join(texts)

## Prepara o input do modelo

In [5]:
def prepare_features(row):
    vaga_text = preprocess_text(combine_vaga_info(row))
    curriculo_text = preprocess_text(str(row.get('cv_pt', '')))
    return vaga_text + ' ' + curriculo_text

## Executa o treino do modelo

In [6]:
def train_model(df):
    # Carregar e preparar dados
    df['match'] = df['perfil_compativel'].map({'sim': 1, 'não': 0})
    df['features'] = df.apply(prepare_features, axis=1)
    
    # Verificar se há dados válidos
    if df['features'].str.strip().eq('').all():
        raise ValueError("Nenhum texto válido encontrado após pré-processamento.")
    
    # Criar vetores TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['features'])
    y = df['match']
    
    # Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Treinar modelo
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Avaliar modelo
    y_pred = model.predict(X_test)
    print(f"Acurácia: {accuracy_score(y_test, y_pred):.2f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.2f}")

    # Salvar modelo e vetorizador
    joblib.dump(model, "model_path/model.pkl")
    joblib.dump(vectorizer, "model_path/vectorizer.pkl")
    
    return model, vectorizer

In [8]:
inicio = time.time()

model, vectorizer = train_model(df)

fim = time.time()
print(f"Tempo de treino: {fim - inicio:.2f} segundos\n")

Acurácia: 0.72
F1-Score: 0.83
Tempo de treino: 111.28 segundos



## Testando o predict do modelo

In [9]:
model = joblib.load("model_path/model.pkl")
vectorizer = joblib.load("model_path/vectorizer.pkl")

In [13]:
def predict_match(vaga_data, curriculo_text):
    """Faz a predição de match para um novo par vaga-currículo."""
    # Preparar features
    vaga_text = preprocess_text(combine_vaga_info(vaga_data))
    curriculo_text = preprocess_text(curriculo_text)
    combined_text = vaga_text + ' ' + curriculo_text
    
    # Verificar se há texto válido
    if not combined_text.strip():
        return 0.0
    
    # Transformar em vetores TF-IDF
    X = vectorizer.transform([combined_text])
    
    # Prever probabilidade
    probability = model.predict_proba(X)[0][1]  # Probabilidade da classe positiva (match)
    return round(probability * 100, 2)

In [14]:
vaga_data = {
    'titulo_vaga': 'Consultor SAP HR Pleno',
    'principais_atividades': 'Implementação e suporte de sistemas SAP HR',
    'competencia_tecnicas_e_comportamentais': 'Experiência em SAP HR',
    'tipo_contratacao': 'PJ/Autônomo',
    'vaga_especifica_para_pcd': 'não',
    'nivel profissional': 'Pleno',
    'nivel_academico': 'Ensino Superior Completo',
    'nivel_ingles': 'Técnico'
}

curriculo_text = """
assistente administrativo
santosbatista
itapecerica da serra/sp
29 anos ▪ brasileiro ▪ casado
formação acadêmica
 bacharel - ciências contábeis
centro universitário ítalo brasileiro
jul/2015 - dez/2018
 graduação - gestão financeira
centro universitário anhanguera
jan/2013 - dez/2014
habilidades
 contas a pagar e receber
 excel avançado
 indicadores kpi’s
 notas fiscais, cfop’s
 fechamento contábil
 emissão de boletos
 guias
 impostos
 budget
 controladoria
 sistemas integrados:
totvs;
folha matic;
navision
resumo profissional
profissional com experiência nos departamentos financeiro,
contábil, fiscal e controladoria jurídica. elaboração e análise de
indicadores kpi’s de resultado, relatórios, guias, gestão de
pagamentos, notas fiscais, boletos, fechamento financeiro e
contábil fiscal.
softwares erp protheus, folha matic, navision, elaw e sapiens,
excel avançado, (kpi's, painéis de dashboard e automatização).
"""

# Fazer predição
probability = predict_match(vaga_data, curriculo_text)
print(f"Probabilidade de match: {probability}%")

Probabilidade de match: 80.0%
